In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# --- विजेटहरू (UI Components) ---
domain_input = widgets.Text(
    value='google',
    placeholder='यहाँ डोमेन नाम लेख्नुहोस्',
    description='डोमेन नाम:',
    style={'description_width': 'initial'}
)

extension_dropdown = widgets.Dropdown(
    options=['.com.np', '.org.np', '.edu.np', '.gov.np', '.net.np'],
    value='.com.np',
    description='एक्सटेन्सन:',
    style={'description_width': 'initial'}
)

lookup_button = widgets.Button(
    description="WHOIS जानकारी हेर्नुहोस्",
    button_style='success',
    tooltip='Click to get WHOIS information',
    icon='search'
)

# आउटपुट देखाउने ठाउँ
output_area = widgets.Output()


# --- मुख्य कार्य (Main Function) ---
def get_whois_info(b):
    """बटन क्लिक गर्दा यो function चल्छ।"""
    domain = domain_input.value.strip()
    extension = extension_dropdown.value
    
    # आउटपुट क्षेत्र खाली गर्ने
    with output_area:
        clear_output(wait=True)
        
        if not domain:
            print("त्रुटि: डोमेन नाम खाली हुन सक्दैन।")
            return

        print(f"'{domain}{extension}' को जानकारी खोजिँदैछ, कृपया पर्खनुहोस्...")

        # Step 1: CSRF टोकन र कुकीहरू प्राप्त गर्ने
        session = requests.Session()
        lookup_url = "https://register.com.np/whois-lookup"
        post_url = "https://register.com.np/checkdomain_whois"
        
        try:
            # GET request पठाएर session र token लिने
            response_get = session.get(lookup_url)
            response_get.raise_for_status() # HTTP त्रुटि भएमा error raise गर्ने
            
            soup_get = BeautifulSoup(response_get.text, 'html.parser')
            token = soup_get.find('input', {'name': '_token'}).get('value')
            
            if not token:
                print("त्रुटि: CSRF टोकन फेला परेन। वेबसाइटको संरचना परिवर्तन भएको हुन सक्छ।")
                return

            # Step 2: WHOIS जानकारीको लागि POST request पठाउने
            payload = {
                '_token': token,
                'domainName': domain,
                'domainExtension': extension,
            }
            
            response_post = session.post(post_url, data=payload)
            response_post.raise_for_status()

            # Step 3: आएको HTML लाई parse गर्ने
            soup_post = BeautifulSoup(response_post.text, 'html.parser')
            
            # यदि डोमेन दर्ता छैन भने, सोही अनुसारको सन्देश देखाउने
            not_registered_msg = soup_post.find(
                'div', 
                class_='alert-danger'
            )
            if not_registered_msg and "is not registered" in not_registered_msg.text:
                print(f"जानकारी: डोमेन '{domain}{extension}' दर्ता गरिएको छैन।")
                return

            # जानकारी भएको table खोज्ने
            whois_table = soup_post.find('table', class_='table-bordered')

            if not whois_table:
                print("त्रुटि: WHOIS जानकारीको तालिका फेला परेन।")
                return

            # तालिकाबाट डाटा निकालेर dictionary मा राख्ने
            whois_info = {}
            rows = whois_table.find_all('tr')
            for row in rows:
                columns = row.find_all('td')
                if len(columns) == 2:
                    key = columns[0].text.strip()
                    value = columns[1].text.strip()
                    whois_info[key] = value

            # Step 4: जानकारीलाई राम्रोसँग प्रदर्शन गर्ने (Pandas DataFrame प्रयोग गरेर)
            if whois_info:
                df = pd.DataFrame(list(whois_info.items()), columns=['विवरण', 'जानकारी'])
                clear_output(wait=True) # "खोजिँदैछ" सन्देश हटाउने
                display(HTML(f"<h3>'{domain}{extension}' को WHOIS जानकारी</h3>"))
                display(df)
            else:
                print("कुनै जानकारी फेला परेन।")

        except requests.exceptions.RequestException as e:
            print(f"नेटवर्क त्रुटि: {e}")
        except Exception as e:
            print(f"एउटा अज्ञात त्रुटि भयो: {e}")

# बटनको on_click event मा function लाई जोड्ने
lookup_button.on_click(get_whois_info)

# UI लाई प्रदर्शन गर्ने
print("======== .NP डोमेन WHOIS Lookup ========")
display(widgets.HBox([domain_input, extension_dropdown]))
display(lookup_button)
display(output_area)

======== .NP डोमेन WHOIS Lookup ========


Button(button_style='success', description='WHOIS जानकारी हेर्नुहोस्', icon='search', style=ButtonStyle(), too…

Output()